For this homework, make sure that you format your notbook nicely and cite all sources in the appropriate sections. Programmatically generate or embed any figures or graphs that you need.

Names: __Zhen Guo and Vikram Chowdhary__

## DO THIS

Step 1: Train your own word embeddings
--------------------------------

We chose to use the provided Spooky Author dataset. It contains text from works of fiction written by "spooky authors" of the public domain - Edgar Allan poe, HP Lovecraft, and Mary Shelley. The features in this dataset are:
- id - a unique identifier for each sentence
- text - some text written by one of the authors
- author - the author of the sentence (EAP: Edgar Allan Poe, HPL: HP Lovecraft; MWS: Mary Wollstonecraft Shelley)
The training portion of this dataset has 19579 texts, and the testing portion has 8392

Describe what data set you have chosen to compare and contrast with the your chosen provided dataset. Make sure to describe where it comes from and it's general properties.

The dataset we selected was found on Kaggle, and consists of 50,000 IMDB reviews. https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [6]:
# import your libraries here
import pandas as pd
import nltk
import re
from nltk.stem import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import nltk
nltk.download('wordnet')
# !pip install gensim

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/vikramc18/nltk_data...


True

### 0) Pre-processing and text-normalization

The following pre-processing steps are inspired from https://towardsdatascience.com/text-normalization-for-natural-language-processing-nlp-70a314bfa646.

We also pre-processed data so that it begins with < s> tokens (and ends with < /s> tokens). Inspired from answer: https://stackoverflow.com/questions/37605710/tokenize-a-paragraph-into-sentence-and-then-into-words-in-nltk

In [2]:
# normalize text to regular expression
# code from https://gist.github.com/yamanahlawat/4443c6e9e65e74829dbb6b47dd81764a

replacement_patterns = [
  (r'won\'t', 'will not'),
  (r'can\'t', 'cannot'),
  (r'i\'m', 'i am'),
  (r'ain\'t', 'is not'),
  (r'(\w+)\'ll', '\g<1> will'),
  (r'(\w+)n\'t', '\g<1> not'),
  (r'(\w+)\'ve', '\g<1> have'),
  (r'(\w+)\'s', '\g<1> is'),
  (r'(\w+)\'re', '\g<1> are'),
  (r'(\w+)\'d', '\g<1> would')
]

patterns = [(re.compile(regex), repl) for (regex, repl) in replacement_patterns]

def replace(text):
    s = text
    for (pattern, repl) in patterns:
        s = re.sub(pattern, repl, s)
    return s

In [3]:
def process_text(text):
    """
    Process the paragram so it is tokenized into sentences, 
    each sentence start with <s> end withh </s>, words are tokenized and normalized for each sentence.
    """
    sent_text = nltk.sent_tokenize(text) # this gives us a list of sentences
    
    # now loop over each sentence and tokenize it separately
    s = []
    for sentence in sent_text:
        # regualr expression
        sentence = replace(sentence)
        # tokenize sentence
        tokenized_text = nltk.word_tokenize(sentence)

        # lematizing and stemming words
        ps = SnowballStemmer("english")
        lemmatizer = WordNetLemmatizer()
        
        new_sent = ['<s>']
        for word in tokenized_text:
            # now remove punctuation
            if not word.isalpha():
                continue           
            # stemming:
            word = ps.stem(word)
            # lemmatizing
            word = lemmatizer.lemmatize(word)
            new_sent.append(word)

        # add begin and end
        new_sent.append('</s>')
        s = s + new_sent
    return s

def process_data(series):
    # returns text in this format:
    # data = [['this', 'is', 'the', 'first', 'sentence', 'for', 'word2vec'],
    # 			['this', 'is', 'the', 'second', 'sentence'],
    # 			['yet', 'another', 'sentence'],
    # 			['one', 'more', 'sentence'],
    # 			['and', 'the', 'final', 'sentence']]
    sentences = []
    for _,row in series.items():
        sentences.append(process_text(row))
    
    return sentences

In [8]:
nltk.download('omw-1.4')

# Read the file and prepare the training data 
# so that it is in the following format
spooky_authors_train = pd.read_csv('./spooky-author-identification/train.csv')
spooky_authors_test = pd.read_csv('./spooky-author-identification/test.csv')
df_imdb = pd.read_csv('IMDB_dataset.csv')

given_data_train = process_data(spooky_authors_train['text'])
given_data_test = process_data(spooky_authors_test['text'])
our_data = process_data(df_imdb['review'])

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/vikramc18/nltk_data...


In [9]:
# save the variables in case leter use
# should be comment out before submission
import pickle

# with open('our_data.pickle', 'wb') as f:
#     pickle.dump(our_data, f)

with open('our_data.pickle', 'rb') as file: 
    # Call load method to deserialze
    out_data = pickle.load(file)

### a) Train embeddings on GIVEN dataset

In [10]:
from gensim.models import Word2Vec

# The dimension of word embedding. 
# This variable will be used throughout the program
# you may vary this as you desire
EMBEDDINGS_SIZE = 200

# Train the Word2Vec model from Gensim. 
# Below are the hyperparameters that are most relevant. 
# But feel free to explore other 
# options too:
# sg = 1
# window = 5
# size = EMBEDDINGS_SIZE
# min_count = 1
# train model on spooky authors training data
model = Word2Vec(sentences = given_data_train, 
                 vector_size = EMBEDDINGS_SIZE,
                 sg = 1, 
                 window = 5,  
                 min_count = 1)

In [11]:
# if you save your Word2Vec as the variable model, this will 
# print out the vocabulary size
# THIS DOES NOT WORK?
# print('Vocab size {}'.format(len(model.wv.vocab)))
# https://stackoverflow.com/questions/35596031/gensim-word2vec-find-number-of-words-in-vocabulary
print('Vocab size {}'.format(len(model.wv)))

Vocab size 14996


In [12]:
# You can save file in txt format, then load later if you wish.
model.wv.save_word2vec_format('embeddings.txt', binary=False)

### b) Train embedding on YOUR dataset

In [13]:
# then do a second data set
# given data is roughly 70/30 train/test
our_data_train = our_data[:35000]
our_data_test = our_data[35000:]
our_model = Word2Vec(sentences = our_data_train, 
                     vector_size = EMBEDDINGS_SIZE,
                     sg = 1, 
                     window = 5,  
                     min_count = 1)

In [14]:
print('Vocab size {}'.format(len(our_model.wv)))

Vocab size 56862


In [15]:
# You can save file in txt format, then load later if you wish.
our_model.wv.save_word2vec_format('imdb_embeddings.txt', binary=False)

What text-normalization and pre-processing did you do and why? __Sentence begin and end, regular expression expansion, tokenization, punctuation removing, lemmatizing and stemming__

Step 2: Evaluate the differences between the word embeddings
----------------------------

(make sure to include graphs, figures, and paragraphs with full sentences)

In [ ]:
# model.wv.index_to_key
# get 10 most common and uncommon words/vectors? plot difference between them?

In [ ]:
model.wv.word_index

##Write down your analysis:

Cite your sources:
-------------

Step 3: Feedforward Neural Language Model
--------------------------

# a) First, encode  your text into integers

In [16]:
# Importing utility functions from Keras
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import Embedding

# The size of the ngram language model you want to train
# change as needed for your experiments
N_GRAM = 3 

# Initializing a Tokenizer
# It is used to vectorize a text corpus. Here, it just creates a mapping from 
# word to a unique index. (Note: Indexing starts from 0)
# Example:
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(data)
# encoded = tokenizer.texts_to_sequences(data)


In [18]:
spooky_tokenizer = Tokenizer()
imdb_tokenizer = Tokenizer()

# spooky authors data
# use out own tokenizer
spooky_train_list = given_data_train
spooky_tokenizer.fit_on_texts(spooky_train_list)
spooky_train_encoded = spooky_tokenizer.texts_to_sequences(spooky_train_list)

# # our data
imdb_train_list = list(df_imdb['review'].values)[:35000]
imdb_tokenizer.fit_on_texts(imdb_train_list)
imdb_train_encoded = imdb_tokenizer.texts_to_sequences(imdb_train_list)

### b) Next, prepare your sequences from text

#### Fixed ngram based sequences 

In [19]:
import itertools

In [20]:

def generate_ngram_training_samples(ngram_len: int, data: list) -> list:
    '''
    Takes the encoded data (list of lists) and 
    generates the training samples out of it.
    Parameters:
    up to you, we've put in what we used
    but you can add/remove as needed
    return: 
    list of lists in the format [[x1, x2, ... , x(n-1), y], ...]
    '''
    # TODO: does this make sense????

    combined_text = list(itertools.chain.from_iterable(data))
    final_ngrams = []
    for idx in range(len(combined_text) - ngram_len + 1):
        ngram_list = list(combined_text[idx:idx+ngram_len])
        final_ngrams.append(ngram_list)
        
    return final_ngrams


### c) Then, split the sequences into X and y and create a Data Generator

In [21]:
# Note here that the sequences were in the form: 
# sequence = [x1, x2, ... , x(n-1), y]
# We still need to separate it into [[x1, x2, ... , x(n-1)], ...], [y1, y2, ...]
def split_ngrams(ngram_list: list) -> list:
    x = [] #those are the context words that we need to get embeddings for
    y = []
    for ngram in ngram_list:
        y.append(ngram[-1])
        x.append(ngram[:-1])
    return x, y

In [22]:
import string

def read_embeddings(text, embeddings, tokenizer):
    '''Loads and parses embeddings trained in earlier.
    Parameters and return values are up to you.
    
    I updated this function so that it takes a list of words as input,
    instead of the raw list of list.
    '''
    
    # you may find generating the following two dicts useful:
    # word to embedding : {'the':[0....], ...}
    # index to embedding : {1:[0....], ...} 
    # use your tokenizer's word_index to find the index of
    # a given word
    word_to_embedding = dict()
    index_to_embedding = dict()
    tok_w_i = tokenizer.word_index
    for word in text:
        # since we already pre processed data, we no longer need to transform
        if word not in word_to_embedding.keys():
            word_to_embedding[word] = embeddings.wv[word]
            index_to_embedding[tok_w_i[word]] = embeddings.wv[word]
                
    return word_to_embedding, index_to_embedding

In [23]:
a, b = read_embeddings(spooky_train_list[0], model, spooky_tokenizer)

In [184]:
import numpy as np

def data_generator(X: list, y: list, num_sequences_per_batch: int, embeddings, tokenizer) -> (list,list):
    '''
    Returns data generator to be used by feed_forward
    https://wiki.python.org/moin/Generators
    https://realpython.com/introduction-to-python-generators/
    
    Yields batches of embeddings and labels to go with them.
    Use one hot vectors to encode the labels 
    (see the to_categorical function)
    
    generator uses yield instead of return
    I don't quit understand what num_sequence_per_batch is...
    Also, only have a vague idea on how this function can be used,
    I would suggest modifying it after we decide how to impliment leaning
    
    '''
    # IDEA: yield num_sequences_per_batch of X, and the same number of y
    # transform y to one hot encodings
    # assume X, y are lists of text/words/whatever comes out of split_ngrams
    cur_idx = 0
    tok_w_i = tokenizer.word_index
    #y = to_categorical(y, num_classes=len(tok_w_i), dtype='float32')
    
    while cur_idx <= len(X) - num_sequences_per_batch:
        X_temp = X[cur_idx:cur_idx + num_sequences_per_batch]
        y_temp = y[cur_idx:cur_idx + num_sequences_per_batch]
        #y_out = y[cur_idx:cur_idx + num_sequences_per_batch]
        
        # assuming below version of embeddings, one hot encodings is correct
        # otherwise keras.preprocessing has a one_hot function that can be used
        X_out = []
        y_out = []
        
        for idx in range(len(X_temp)):
            # get embeddings for words in X
            w_2_e, i_2_e = read_embeddings(X_temp[idx], embeddings, tokenizer)
            temp_embeddings = np.array(list(w_2_e.values()))
            flat_emb = [item for sublist in temp_embeddings for item in sublist]
            X_out.append(np.array(flat_emb))
            # X_out += list(w_2_e.values())
            
            #y_out.append(tok_w_i[y_temp[idx]])
#             w2e_y, i2e_y = read_embeddings(y_temp[idx], embeddings, tokenizer)
#             y_out.append(list(w2e_y.values()))
            # get one-hot for words in y
            word_y = y_temp[idx]
            y_vect = [0] * len(tok_w_i)
            y_vect[tok_w_i[word_y]] = 1
            y_out.append(y_vect)
        
        cur_idx += num_sequences_per_batch
#         y_out = to_categorical(y_out)
        yield np.array(X_out), np.array(y_out)
        
        
#     # let us get the embeddings for list X
#     # this should be a matrix of embeddings of words list in X
#     # each word list has its embeddings
#     for idx in range(len(y)):
#         # these get us embeddings of x in each word list in X
#         w_2_e, i_2_e = read_embeddings(X[idx], embeddings, tokenizer)

#         # get one hot for y
#         word_y = y[i]
#         tok_w_i = tokenizer.word_index # get index of y
#         y_vect = [0] * len(tok_w_i) # initialize zeros
#         y_vect[tok_w_i[word_y]] = 1 # set y to 1
        
#         yield w_2_e, y_vect



In [185]:
# Examples
spooky_n_gram_temp = generate_ngram_training_samples(N_GRAM, spooky_train_list)
X, y = split_ngrams(spooky_n_gram_temp)
# initialize data_generator
num_sequences_per_batch = 128 # this is the batch size
steps_per_epoch = len(spooky_train_list)//num_sequences_per_batch  # Number of batches per epoch
train_generator = data_generator(X, y, num_sequences_per_batch, model, spooky_tokenizer)

sample=next(train_generator) # this is how you get data out of generators
# sample[0].shape # (batch_size, (n-1)*EMBEDDING_SIZE)  (128, 200)
# sample[1].shape   # (batch_size, |V|) to_categorical

In [156]:
sample[0][0].shape

(400,)

In [154]:
sample[0].shape

(128, 400)

In [155]:
sample[1].shape

(128, 14996)

In [211]:
len(spooky_n_gram_temp)

563445

In [216]:
X[750:770]

[['sound', 'were'],
 ['were', 'hideous'],
 ['hideous', 'for'],
 ['for', 'they'],
 ['they', 'were'],
 ['were', 'not'],
 ['not', 'but'],
 ['but', 'that'],
 ['that', 'they'],
 ['they', 'held'],
 ['held', 'vibrat'],
 ['vibrat', 'suggest'],
 ['suggest', 'noth'],
 ['noth', 'on'],
 ['on', 'this'],
 ['this', 'globe'],
 ['globe', 'of'],
 ['of', 'earth'],
 ['earth', 'and'],
 ['and', 'that']]

In [219]:
t = [item for sublist in X[750:770] for item in sublist]
z = read_embeddings(t, model, spooky_tokenizer)

In [227]:
len(set(t))

18

In [221]:
len(t)

40

In [220]:
len(z)

2

In [228]:
z[0]['vibrat']

array([-6.94103632e-03,  1.95030533e-02,  6.32904768e-02,  7.40178302e-02,
        1.62061885e-01, -1.43986896e-01,  4.29557497e-03,  2.89301962e-01,
       -3.58481929e-02,  1.11943604e-02, -4.70941179e-02, -1.38665617e-01,
        9.51787010e-02,  2.75724661e-02, -3.38388719e-02, -6.19822964e-02,
       -6.52220547e-02, -2.38603372e-02, -2.11231057e-02, -2.91984290e-01,
        9.67712104e-02, -3.77771351e-03, -5.71797490e-02,  3.44971046e-02,
       -1.56434044e-01, -7.04550594e-02, -1.82312839e-02, -3.55994105e-02,
       -9.22890455e-02, -3.03443875e-02,  1.47319928e-01, -1.83679443e-02,
        7.14467168e-02, -3.82174030e-02,  2.75653284e-02,  6.50845021e-02,
        1.37879878e-01,  7.40505755e-02, -6.16799667e-03, -1.29875645e-01,
       -1.17864028e-01, -7.71678314e-02, -1.89000368e-02,  9.15728137e-02,
        1.40934199e-01, -2.41736489e-04,  2.34490559e-02, -4.46258113e-02,
       -1.74920894e-02,  1.10793360e-01,  7.83110410e-02, -8.39373395e-02,
       -1.21704631e-01, -

In [225]:
len(z[1])

18

### d) Train your models

In [170]:
# code to train a feedforward neural language model 
# on a set of given word embeddings
# make sure not to just copy + paste to train your two models
# https://pyimagesearch.com/2021/05/06/implementing-feedforward-neural-networks-with-keras-and-tensorflow/
# Define the model architecture using Keras Sequential API
spooky_nn_model = Sequential()
# model.add(Dense(256, input_shape=(784,), activation="sigmoid"))
spooky_nn_model.add(Dense(256, input_shape=((N_GRAM-1)*EMBEDDINGS_SIZE,), activation='sigmoid'))
# model.add(Dense(128, activation="sigmoid"))
spooky_nn_model.add(Dense(128, activation='sigmoid'))
# model.add(Dense(10, activation="softmax"))
spooky_nn_model.add(Dense(14996, activation='softmax'))
# sgd = SGD(0.01)
# model.compile(loss="categorical_crossentropy", optimizer=sgd,
# 	metrics=["accuracy"])
spooky_nn_model.compile(loss='categorical_crossentropy', optimizer='sgd',metrics=['accuracy'])


In [204]:
sample1=next(train_generator)
sample2=next(train_generator)
sample3=next(train_generator)
sample4=next(train_generator)
sample5=next(train_generator)
sample6=next(train_generator)
sample7=next(train_generator)

<ipython-input-184-2a6857499b7a>:55: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  yield np.array(X_out), np.array(y_out)


In [207]:
sample5[0].shape

(128, 400)

In [206]:
sample6[0].shape

(128,)

In [205]:
sample7[0].shape

(128, 400)

In [195]:
# Start training the model
spooky_nn_model.fit(x=train_generator, 
                    steps_per_epoch=7,
                    epochs=1)

6/7 [========================>.....] - ETA: 0s - loss: 9.4630 - accuracy: 0.0703

<ipython-input-184-2a6857499b7a>:55: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  yield np.array(X_out), np.array(y_out)


InvalidArgumentError: Graph execution error:

TypeError: `generator` yielded an element that could not be converted to the expected type. The expected type was float32, but the yielded element was [array([ 0.0491656 , -0.00917797,  0.04115955,  0.02218254,  0.08492419,
        -0.04591661, -0.00552306,  0.11725595, -0.01674656,  0.00623285,
        -0.06040242, -0.04545799,  0.02372918,  0.00628725, -0.02771797,
        -0.02125333, -0.05888021, -0.00688055,  0.0052118 , -0.1242099 ,
         0.04233667, -0.02455211, -0.02521667,  0.02106701, -0.05573503,
        -0.03598316,  0.00811036,  0.0108192 , -0.0743475 , -0.02147009,
         0.06824736,  0.00315619,  0.0253255 , -0.00056603,  0.06028323,
         0.01979605,  0.08739368,  0.03994142,  0.01428901, -0.05175827,
        -0.02945219, -0.06165364, -0.00355854,  0.06936429,  0.08966168,
        -0.03171358, -0.01157956, -0.05410049, -0.00127431,  0.06797805,
         0.04810951,  0.00238508, -0.04229266, -0.09105227, -0.03555815,


### e) Generate Sentences

In [ ]:
# generate a sequence from the model
def generate_seq(model: Sequential, 
                 tokenizer: Tokenizer, 
                 seed: list, 
                 n_words: int):
    '''
    Parameters:
        model: your neural network
        tokenizer: the keras preprocessing tokenizer
        seed: [w1, w2, w(n-1)]
        n_words: generate a sentence of length n_words
    Returns: string sentence
    '''
    pass

### f) Compare your generated sentences

Sources Cited
----------------------------
